There are obviously two parts to this problem: first, calculate $\binom{n}{k}$ mod $p$ for all primes in range; secondly, apply Chinese Remainder Theorem to each prime triple. The second part is very straightforward.

For the first part, we first use Legendre's formula to calculate $\nu_p(\binom{n}{k})$. If $\nu_p > 0$, the remainder is simply zero. Otherwise, let

$$\mathfrak{m}(n,\,p) = \frac{n!}{p^{\nu_p(n!)}},$$

with $n = q \cdot p + r$ we have

$$\mathfrak{m}(n,\,p) \equiv ((p-1)!)^q \cdot r! \cdot \frac{\prod_{k=1}^q kp}{p^{\nu_p(n!)}} \equiv (-1)^q \cdot r! \cdot \mathfrak{m}(k,\,p),$$

where we have used Wilson's theorem. Hence $\mathfrak{m}$ can be computed recursively.

In [1]:
#!/usr/bin/env python3

import functools
import sys

import primesieve


N = 10 ** 18
K = 10 ** 9
PRIME_RANGE = (1000, 5000)

sys.setrecursionlimit(PRIME_RANGE[1] * 2)


def binom_nk_mod(p):
    def factorial_vp(n):
        vp = 0
        while n >= p:
            n //= p
            vp += n
        return vp

    # Calculates n!/p^vp(n!) mod p.
    @functools.lru_cache(maxsize=None)
    def factorial_modp(n):
        if n in (0, 1):
            return 1
        if n == p - 1:
            return -1
        if n < p:
            return n * factorial_modp(n - 1) % p
        # Break n! into n!/(p^k k!) and p^k k! (i.e., p * 2p * ... * kp)
        # where k = floor(n/p). Use Wilson's formula plus recursion.
        q, r = divmod(n, p)
        return (1 if q % 2 == 0 else -1) * factorial_modp(r) * factorial_modp(q) % p

    vp = factorial_vp(N) - factorial_vp(K) - factorial_vp(N - K)
    if vp > 0:
        return 0
    return (
        factorial_modp(N)
        * pow(factorial_modp(K), -1, p)
        * pow(factorial_modp(N - K), -1, p)
        % p
    )


def chinese_remainder(p1r1, p2r2, p3r3):
    p1, r1 = p1r1
    p2, r2 = p2r2
    p3, r3 = p3r3
    s = 0
    if r1 != 0:
        p2p3 = p2 * p3
        s += r1 * p2p3 * pow(p2p3, -1, p1)
    if r2 != 0:
        p3p1 = p3 * p1
        s += r2 * p3p1 * pow(p3p1, -1, p2)
    if r3 != 0:
        p1p2 = p1 * p2
        s += r3 * p1p2 * pow(p1p2, -1, p3)
    return s % (p1 * p2 * p3)


def main():
    primes = primesieve.primes(PRIME_RANGE[0] + 1, PRIME_RANGE[1] - 1)
    n_primes = len(primes)
    p_remainders = [(p, binom_nk_mod(p)) for p in primes]
    total = 0
    for i in range(n_primes):
        print(f"p: {primes[i]} ({i+1}/{n_primes})")
        for j in range(i + 1, n_primes):
            for k in range(j + 1, n_primes):
                total += chinese_remainder(
                    p_remainders[i], p_remainders[j], p_remainders[k]
                )
    print(total)


if __name__ == "__main__":
    main()


p: 1009 (1/501)
p: 1013 (2/501)
p: 1019 (3/501)
p: 1021 (4/501)
p: 1031 (5/501)
p: 1033 (6/501)
p: 1039 (7/501)
p: 1049 (8/501)
p: 1051 (9/501)
p: 1061 (10/501)
p: 1063 (11/501)
p: 1069 (12/501)
p: 1087 (13/501)
p: 1091 (14/501)
p: 1093 (15/501)
p: 1097 (16/501)
p: 1103 (17/501)
p: 1109 (18/501)
p: 1117 (19/501)
p: 1123 (20/501)
p: 1129 (21/501)
p: 1151 (22/501)
p: 1153 (23/501)
p: 1163 (24/501)
p: 1171 (25/501)
p: 1181 (26/501)
p: 1187 (27/501)
p: 1193 (28/501)
p: 1201 (29/501)
p: 1213 (30/501)
p: 1217 (31/501)
p: 1223 (32/501)
p: 1229 (33/501)
p: 1231 (34/501)
p: 1237 (35/501)
p: 1249 (36/501)
p: 1259 (37/501)
p: 1277 (38/501)
p: 1279 (39/501)
p: 1283 (40/501)
p: 1289 (41/501)
p: 1291 (42/501)
p: 1297 (43/501)
p: 1301 (44/501)
p: 1303 (45/501)
p: 1307 (46/501)
p: 1319 (47/501)
p: 1321 (48/501)
p: 1327 (49/501)
p: 1361 (50/501)
p: 1367 (51/501)
p: 1373 (52/501)
p: 1381 (53/501)
p: 1399 (54/501)
p: 1409 (55/501)
p: 1423 (56/501)
p: 1427 (57/501)
p: 1429 (58/501)
p: 1433 (59/501)
p: 143